# Bermudan Swaptions example

This notebook is just a reorganized version of examples in https://github.com/lballabio/QuantLib-SWIG/blob/master/Python/examples/bermudan-swaption.py
The difference  is just exposing critical parameters to allow people not used to reading python code to play with them with small changes in the levels to make it closer to current reality.

In [1]:
# importing libraries

# # Bermudan swaptions
#
# Copyright (&copy;) 2004, 2005, 2006, 2007 StatPro Italia srl
# 2022 Wojciech Ślusarski
#
# This file is part of QuantLib, a free-software/open-source library
# for financial quantitative analysts and developers - https://www.quantlib.org/
#
# QuantLib is free software: you can redistribute it and/or modify it under the
# terms of the QuantLib license.  You should have received a copy of the
# license along with this program; if not, please email
# <quantlib-dev@lists.sf.net>. The license is also available online at
# <https://www.quantlib.org/license.shtml>.
#
# This program is distributed in the hope that it will be useful, but WITHOUT
# ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
# FOR A PARTICULAR PURPOSE.  See the license for more details.

import QuantLib as ql
import pandas as pd

In [2]:
# ### Setup

todaysDate = ql.Date(31, ql.October, 2022)

logNormalVolatility = 0.30

atmRate = 0.075

notional = 10000

# spread for the otm Swap above atm in bps
upper_spread = 100

# spread for the otm swap below atm in bps
low_spread = -100

# swaption types as receiver swaptions
payFixed = ql.Swap.Receiver

# Swaption tenor in years
swaption_tenor = 5

## Setup the pricing engines

In [3]:
ql.Settings.instance().evaluationDate = todaysDate
calendar = ql.Poland()
settlementDate = calendar.advance(todaysDate, ql.Period('2d'), ql.Following)

bps = 1/10000


def calibrate(model, helpers, l, name):
    print("Model: %s" % name)

    method = ql.Simplex(l)
    model.calibrate(helpers, method, ql.EndCriteria(1000, 250, 1e-7, 1e-7, 1e-7))

    print("Parameters: %s" % model.params())

    totalError = 0.0
    data = []
    for swaption, helper in zip(swaptionVols, helpers):
        maturity, length, vol = swaption
        NPV = helper.modelValue()
        implied = helper.impliedVolatility(NPV, 1.0e-4, 1000, 0.05, 0.50)
        error = implied - vol
        totalError += abs(error)
        data.append((maturity, length, vol, implied, error))
    averageError = totalError / len(helpers)

    print(pd.DataFrame(data, columns=["maturity", "length", "volatility", "implied", "error"]))

    print("Average error: %.4f" % averageError)


# ### Market data

swaptionVols = [
    # maturity,          length,             volatility
    (ql.Period(i, ql.Years), ql.Period(swaption_tenor - i, ql.Years), logNormalVolatility)
    for i in range(1, swaption_tenor)
]

# This is a flat yield term structure.

rate = ql.QuoteHandle(ql.SimpleQuote(atmRate))
termStructure = ql.YieldTermStructureHandle(ql.FlatForward(settlementDate, rate, ql.Actual365Fixed()))

# Define the ATM/OTM/ITM swaps:

swapEngine = ql.DiscountingSwapEngine(termStructure)

fixedLegFrequency = ql.Semiannual
fixedLegTenor = ql.Period(6, ql.Months)
fixedLegConvention = ql.Unadjusted
floatingLegConvention = ql.ModifiedFollowing
fixedLegDayCounter = ql.ActualActual(ql.ActualActual.ISDA)
floatingLegFrequency = ql.Semiannual
floatingLegTenor = ql.Period(6, ql.Months)

fixingDays = 2
index = ql.Wibor(ql.Period('6M'), termStructure)
floatingLegDayCounter = index.dayCounter()

swapStart = calendar.advance(settlementDate, 0, ql.Years, floatingLegConvention)
swapEnd = calendar.advance(swapStart, swaption_tenor, ql.Years, floatingLegConvention)

fixedSchedule = ql.Schedule(
    swapStart,
    swapEnd,
    fixedLegTenor,
    calendar,
    fixedLegConvention,
    fixedLegConvention,
    ql.DateGeneration.Forward,
    False,
)
floatingSchedule = ql.Schedule(
    swapStart,
    swapEnd,
    floatingLegTenor,
    calendar,
    floatingLegConvention,
    floatingLegConvention,
    ql.DateGeneration.Forward,
    False,
)

dummy = ql.VanillaSwap(
    payFixed, notional, fixedSchedule, 0.0, fixedLegDayCounter, floatingSchedule, index, 0.0, floatingLegDayCounter
)
dummy.setPricingEngine(swapEngine)
atmRate = dummy.fairRate()
otmRate = atmRate + low_spread * bps
itmRate = atmRate + upper_spread * bps

atmSwap = ql.VanillaSwap(
    payFixed, notional, fixedSchedule, atmRate, fixedLegDayCounter,
    floatingSchedule, index, 0.0, floatingLegDayCounter
)

otmSwap = ql.VanillaSwap(
    payFixed, notional, fixedSchedule, otmRate, fixedLegDayCounter,
    floatingSchedule, index, 0.0, floatingLegDayCounter
)

itmSwap = ql.VanillaSwap(
    payFixed, notional, fixedSchedule, itmRate, fixedLegDayCounter,
    floatingSchedule, index, 0.0, floatingLegDayCounter
)

atmSwap.setPricingEngine(swapEngine)
otmSwap.setPricingEngine(swapEngine)
itmSwap.setPricingEngine(swapEngine)

helpers = [
    ql.SwaptionHelper(
        maturity,
        length,
        ql.QuoteHandle(ql.SimpleQuote(vol)),
        index,
        index.tenor(),
        index.dayCounter(),
        index.dayCounter(),
        termStructure,
    )
    for maturity, length, vol in swaptionVols
]

times = {}
for h in helpers:
    for t in h.times():
        times[t] = 1
times = sorted(times.keys())

grid = ql.TimeGrid(times, 30)


HWmodel = ql.HullWhite(termStructure)
HWmodel2 = ql.HullWhite(termStructure)
BKmodel = ql.BlackKarasinski(termStructure)

# ### Calibrations

for h in helpers:
    h.setPricingEngine(ql.JamshidianSwaptionEngine(HWmodel))
calibrate(HWmodel, helpers, 0.05, "Hull-White (analytic formulae)")

for h in helpers:
    h.setPricingEngine(ql.TreeSwaptionEngine(HWmodel2, grid))
calibrate(HWmodel2, helpers, 0.05, "Hull-White (numerical calibration)")

for h in helpers:
    h.setPricingEngine(ql.TreeSwaptionEngine(BKmodel, grid))
calibrate(BKmodel, helpers, 0.05, "Black-Karasinski (numerical calibration)")



Model: Hull-White (analytic formulae)
Parameters: [ 0.0639488; 0.0255733 ]
  maturity length  volatility   implied     error
0       1Y     4Y         0.3  0.298419 -0.001581
1       2Y     3Y         0.3  0.299311 -0.000689
2       3Y     2Y         0.3  0.300407  0.000407
3       4Y     1Y         0.3  0.301914  0.001914
Average error: 0.0011
Model: Hull-White (numerical calibration)
Parameters: [ 0.0695293; 0.0259656 ]
  maturity length  volatility   implied     error
0       1Y     4Y         0.3  0.292209 -0.007791
1       2Y     3Y         0.3  0.299734 -0.000266
2       3Y     2Y         0.3  0.302778  0.002778
3       4Y     1Y         0.3  0.305182  0.005182
Average error: 0.0040
Model: Black-Karasinski (numerical calibration)
Parameters: [ 0.0487168; 0.339991 ]
  maturity length  volatility   implied     error
0       1Y     4Y         0.3  0.295127 -0.004873
1       2Y     3Y         0.3  0.300853  0.000853
2       3Y     2Y         0.3  0.301983  0.001983
3       4Y     1Y 

## Perform pricing with different models
Prices displayed in bps of notional as a spot premium

In [4]:
# ### Price Bermudan swaptions on defined swaps

bermudanDates = [d for d in fixedSchedule][:-1]
exercise = ql.BermudanExercise(bermudanDates)

atmSwaption = ql.Swaption(atmSwap, exercise)
otmSwaption = ql.Swaption(otmSwap, exercise)
itmSwaption = ql.Swaption(itmSwap, exercise)

data = []


# +
atmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel, 50))
otmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel, 50))
itmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel, 50))

data.append(("HW analytic", itmSwaption.NPV(), atmSwaption.NPV(), otmSwaption.NPV()))

# +
atmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel2, 50))
otmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel2, 50))
itmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel2, 50))

data.append(("HW numerical", itmSwaption.NPV(), atmSwaption.NPV(), otmSwaption.NPV()))

# +
atmSwaption.setPricingEngine(ql.TreeSwaptionEngine(BKmodel, 50))
otmSwaption.setPricingEngine(ql.TreeSwaptionEngine(BKmodel, 50))
itmSwaption.setPricingEngine(ql.TreeSwaptionEngine(BKmodel, 50))

data.append(("BK numerical", itmSwaption.NPV(), atmSwaption.NPV(), otmSwaption.NPV()))
# -

pd.options.display.float_format = "{:,.2f}".format
df = pd.DataFrame(data, columns=["model", "in-the-money", "at-the-money", "out-of-the-money"])




,model,in-the-money,at-the-money,out-of-the-money
0,HW analytic,599.89,414.74,276.20
1,HW numerical,601.93,416.81,277.89
2,BK numerical,620.97,409.50,249.33


## Display results

In [6]:
display(df)

,model,in-the-money,at-the-money,out-of-the-money
0,HW analytic,599.89,414.74,276.20
1,HW numerical,601.93,416.81,277.89
2,BK numerical,620.97,409.50,249.33


In [7]:
# Prices displayed as a running spread in bps
df.loc[:, ("in-the-money", "at-the-money", "out-of-the-money")] / atmSwap.fixedLegBPS()

,in-the-money,at-the-money,out-of-the-money
0,146.66,101.39,67.52
1,147.15,101.90,67.94
2,151.81,100.11,60.95
